In [27]:
import numpy as np
from pyvis.network import Network

In [28]:
def get_genealogy(G, N):
    parent = np.random.choice(N, (G, N))
    parent.sort(axis=1)
    for n in range(N):
        parent[0, n] = -1
    for g in range(1, G):
        for n in range(N):
            parent[g, n] += (g-1)*N
    return parent


In [29]:
def get_mutations(G, N, p, genealogy):
    r = np.random.rand(G, N)
    mut_counter = 0
    new_mutations = r < p
    mutations = [set() for _ in range(G * N)]
    for g in range(G):
        for n in range(N):
            if new_mutations[g, n]:
                mutations[g*N + n].add(mut_counter)
                mut_counter += 1
                
    for g in range(G):
        for n in range(N):
            if genealogy[g, n] != -1:
                mutations[g*N + n] = mutations[g*N + n].union(mutations[genealogy[g, n]])
    return mutations
    

In [30]:
N = 10
G = N
p = 1
genealogy = get_genealogy(G, N)
mutations = get_mutations(G, N, p, genealogy)

In [37]:
color_map = {0: 'cyan', 1: 'yellow', 2:'orange', 3:'red'}
labels = [f'{g},{n}\n{mutations[g*N + n] or ""}' for g in range(G) for n in range(N)]
net = Network(directed=True, notebook=True, height='1000px', width='1000px', cdn_resources='remote')
net.add_nodes(range(G*N), y=[100*n for g in range(G) for n in range(N)],
                          x=[100*g for g in range(G) for n in range(N)],
                          color=[color_map[min(len(mutations[g*N + n]), 3)] for g in range(G) for n in range(N)],
                          title=labels)
net.add_edges([(int(genealogy[g, n]), g*N+n) for g in range(1, G) for n in range(N)])
net.toggle_physics(False)

In [38]:
net.show('genealogy.html')

genealogy.html


In [33]:
# cluster nodes with same mutations
clusters = {}
for g in range(G):
    for n in range(N):
        key = tuple(sorted(mutations[g*N + n]))
        if key not in clusters:
            clusters[key] = []
        clusters[key].append(g*N + n)

In [34]:
def is_child_mutation(mutation, parent):
    return (len(mutation) == len(parent) - 1) and set(mutation).issubset(set(parent))

In [35]:
cluster_items = list(clusters.items())
cluster_graph = Network(directed=True)
for i, (key, nodes) in enumerate(cluster_items):
    cluster_graph.add_node(i, title=f"{key}\nsize: {len(nodes)}")

for i, (key, nodes) in enumerate(cluster_items):
    for j, (key2, nodes2) in enumerate(cluster_items):
        if i == j:
            continue
        
        if is_child_mutation(key, key2):
            cluster_graph.add_edge(i, j)


In [36]:
cluster_graph.show('cluster.html')

cluster.html


AttributeError: 'NoneType' object has no attribute 'render'

In [35]:
last_generation = mutations[-N:]
# save to file
with open('mutations.txt', 'w') as f:
    for mutation in last_generation:
        f.write(f'{mutation}\n')